# <div style="text-align: center">The Sparks Foundations IoT & Computer Vision intern </div>

## <center>Task 1: Object Detetion  </center>
<div style="text-align: right"> <font size=4>
By: Md. Yeasin Sheikh</font> </div>

### We are using MobilNet-SSD

In [1]:
# packages we need 
import cv2
import numpy as np

In [2]:
print('we are using cv', cv2.__version__)

we are using cv 4.5.1


### List of objects that can be detected using this model

In [3]:
classNames= []
classFilePath = 'data/coco.names'
with open(classFilePath,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

In [4]:
print('Number of objects that can be Detected: ', len(classNames))
print(classNames)

Number of objects that can be Detected:  91
['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothb

## Creating dnn Detection Model 
locating the trained model class path 

In [5]:
configPath = 'data/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'data/frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

<dnn_Model 000002D24D80E550>

## Lets'  test on an image
importing image from assets directory

In [6]:
img = cv2.imread('assets/dogCat.png')
print(img.shape)

(800, 1580, 3)


### This image kinda big, let's Resize it 

In [7]:
# resize a bit 
imgHeight, imgWidth, _ = img.shape
scale= 60
imgH = int (imgHeight *scale /100)
imgW = int (imgWidth * scale /100)
img = cv2.resize(img, (imgW, imgH))

### Lets' declear some variables

In [8]:
rectColor =(244,5,20)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
fontSize = 1
fontColor = (255, 0, 255)

<font size=4> Threshold to detect object

In [9]:
thres = 0.45 # Threshold to detect object
nms_thres = .5 # 0-1 higer value means lower suppress 

In [10]:
classIds, confs, bbox = net.detect(img,confThreshold=thres)
type(confs),type(bbox)

(numpy.ndarray, numpy.ndarray)

# Formating for non-maxima suppression
<font size=4> it removes overlap bounding boxes & keep the most confident ones.</font>
<font size=3> make sure bounding boxex and confident are List of floats, it shouldn't associate with numpy 
    </font>


In [11]:
bbox = list(bbox)
confs = list(np.array(confs).reshape(1, -1)[0])
confs = list(map(float,confs ))

indics = cv2.dnn.NMSBoxes(bbox, confs, thres, nms_thres) # remove overlaps
print('Number of bjects we get: ',len(indics))

Number of bjects we get:  2


<font size=3.5> Take a look on actual image</font>

In [12]:
for ind in indics:
    i = ind[0] # ind returns list we need index only
    box = bbox[i]
    
    cv2.rectangle(img, box, rectColor, 1) #set the rectangle around object
    
    obj_name = classNames[classIds[i][0]-1]
    obj_confidences = round(confs[i]*100, 1)
    print(obj_name, " conf ", str(obj_confidences))
    
    if obj_confidences>56:
        #show accuracy level
        cv2.putText(img, str(obj_confidences), (box[0]+10, box[1]+10), 
                   font, fontSize, fontColor, 1,cv2.LINE_AA)
    
        #putting name of object 
        cv2.putText(img, obj_name.upper(), (box[0]+70, box[1]+10), 
                   font, fontSize, fontColor,1,cv2.LINE_AA)
    
# Lets show the result 
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

cat  conf  82.1
dog  conf  76.0


# Let's test it on webcam 

In [13]:
import cv2
cam = cv2.VideoCapture(0) # my webcam id
# cam = cv2.VideoCapture('assets/testVideo.mp4') # my webcam id
cam.set(3, 1000) # width of camView
cam.set(4, 800) # height of camView
cam.set(10, 150) # brightness 

True

In [14]:
# we can also make a funtion just for image processing 
while True:
    success, img = cam.read()
    if success:
        classIds, confs, bbox = net.detect(img,confThreshold=thres)
        
        #Formating data
        bbox = list(bbox)
        confs = list(np.array(confs).reshape(1, -1)[0])
        confs = list(map(float,confs ))

        indics = cv2.dnn.NMSBoxes(bbox, confs, thres, nms_thres) # remove overlaps
        
        for ind in indics:
            i = ind[0] # ind returns list we need index only
            box = bbox[i]
    
            cv2.rectangle(img, box, rectColor, 1) #set the rectangle around object
    
            #show accuracy level
            cv2.putText(img, str(round(confs[i]*100, 1)), (box[0]+10, box[1]+10), 
                       font, fontSize, fontColor, 1)
            #putting name of object 
            cv2.putText(img, classNames[classIds[i][0]-1], (box[0]+70, box[1]+30), 
                       font, fontSize, fontColor,1)
        
    
        cv2.imshow("Object detection", img)
        if cv2.waitKey(1) == 27: 
            break  # esc to quit
    
cam.release()
cv2.destroyAllWindows()

# <center> Thanks for your time 😊</center>